<a href="https://colab.research.google.com/github/gaston411/Mi-Proyecto/blob/brendanahir/Kmean_Lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud    import WordCloud
%matplotlib inline
import matplotlib.cm as cm

from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.decomposition import LatentDirichletAllocation as LDA

from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import Normalizer

In [5]:
import spacy

In [6]:
!python -m spacy download es_core_news_sm

2022-07-29 14:19:17.607013: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 7.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('/content/drive/MyDrive/ArchivosCSV/df_parrafosCompletos.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df1 = df['parrafo'].apply(str)
df1

0       I-PROEMIO:\nSr. JUEZ NACIONAL:\nHORTAS MARIA A...
1       II-ANTECEDENTES DE INTERES MEDICO-LEGAL:\nlas ...
2       III-EXAMEN FISICO DEL ACTOR:\nIII-1 PROSOGRAFI...
3       IV-CONSIDERACIONES MEDICO-LEGALES:\nComenta el...
4       VII-CONCLUSIONES.\n1.\nLas secuelas en el tobi...
                              ...                        
4106    V) CONCLUSIONES\nEn conclusion, la Srta. MARIN...
4107    VI) PETITORIO\nSolicito respetuosamente a  que...
4108    IX.- PETITORIO: Conforme lo dicho solicito:\n1...
4109    III.- ESTUDIOS SOLICITADOS\nA.- Fojas RMN homb...
4110    IV.- PUNTOS PERICIALES\nActora\n1.- contestado...
Name: parrafo, Length: 4111, dtype: object

In [9]:
dfnr= pd.DataFrame(df1)
dfnr

,parrafo
0,I-PROEMIO:\nSr. JUEZ NACIONAL:\nHORTAS MARIA A...
1,II-ANTECEDENTES DE INTERES MEDICO-LEGAL:\nlas ...
2,III-EXAMEN FISICO DEL ACTOR:\nIII-1 PROSOGRAFI...
3,IV-CONSIDERACIONES MEDICO-LEGALES:\nComenta el...
4,VII-CONCLUSIONES.\n1.\nLas secuelas en el tobi...
...,...
4106,"V) CONCLUSIONES\nEn conclusion, la Srta. MARIN..."
4107,VI) PETITORIO\nSolicito respetuosamente a que...
4108,IX.- PETITORIO: Conforme lo dicho solicito:\n1...
4109,III.- ESTUDIOS SOLICITADOS\nA.- Fojas RMN homb...


In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopws = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer ('spanish')

In [12]:
nlp= spacy.load('es_core_news_sm')

In [13]:
stop_ws= nlp.Defaults.stop_words
stop_ws

{'a',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'afirmó',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún',
 'alli',
 'allí',
 'alrededor',
 'ambos',
 'ante',
 'anterior',
 'antes',
 'apenas',
 'aproximadamente',
 'aquel',
 'aquella',
 'aquellas',
 'aquello',
 'aquellos',
 'aqui',
 'aquél',
 'aquélla',
 'aquéllas',
 'aquéllos',
 'aquí',
 'arriba',
 'aseguró',
 'asi',
 'así',
 'atras',
 'aun',
 'aunque',
 'añadió',
 'aún',
 'bajo',
 'bastante',
 'bien',
 'breve',
 'buen',
 'buena',
 'buenas',
 'bueno',
 'buenos',
 'cada',
 'casi',
 'cierta',
 'ciertas',
 'cierto',
 'ciertos',
 'cinco',
 'claro',
 'comentó',
 'como',
 'con',
 'conmigo',
 'conocer',
 'conseguimos',
 'conseguir',
 'considera',
 'consideró',
 'consigo',
 'consigue',
 'consiguen',
 'consigues',
 'contigo',
 'contra',
 'creo',
 'cual',
 'cuales',
 'cualquier',
 'cuando',
 'cuanta',
 'cuantas',
 'cuanto',
 'cuantos',
 'cuatro',
 'cuenta',
 'cuál',
 'cuáles',
 'cu

In [14]:
def limpiar_palabras(text: str) -> str:
    '''
    Limpiar palabras en títulos. Solamente deja palabras y espacios en blanco.
    Params:
        **text**:texto a ser limpiado de palabras no desaeadas
    '''
    text = re.sub(r'(I{1,3}|IV|V|VI{1,4}|IX|X)[). -]|[^\w\s]',' ',text)
    text = text.lower().replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') # removemos minúsculas y tildes
    text = re.sub('[^a-zA-Z]',' ', text) #removemos caracteres especiales y numeros.

    text = [word for word in text.split(' ') if word not in stop_ws] #removemos stopwords
    text = ' '.join(text)
    text = [
        i for i in text.split() if len(i) > 3
    ]
    return ' '.join(text)

  

In [15]:
dfnr['seccion_limpia']= dfnr.parrafo.apply(limpiar_palabras)
dfnr

,parrafo,seccion_limpia
0,I-PROEMIO:\nSr. JUEZ NACIONAL:\nHORTAS MARIA A...,proemio juez nacional hortas maria andrea peri...
1,II-ANTECEDENTES DE INTERES MEDICO-LEGAL:\nlas ...,antecedentes interes medico legal encontraba c...
2,III-EXAMEN FISICO DEL ACTOR:\nIII-1 PROSOGRAFI...,examen fisico actor prosografia presenta exame...
3,IV-CONSIDERACIONES MEDICO-LEGALES:\nComenta el...,consideraciones medico legales comenta boiero ...
4,VII-CONCLUSIONES.\n1.\nLas secuelas en el tobi...,conclusiones secuelas tobillo izquierdo presen...
...,...,...
4106,"V) CONCLUSIONES\nEn conclusion, la Srta. MARIN...",conclusiones conclusion srta marina luisa aval...
4107,VI) PETITORIO\nSolicito respetuosamente a que...,petitorio solicito respetuosamente presentado ...
4108,IX.- PETITORIO: Conforme lo dicho solicito:\n1...,petitorio conforme solicito presentada pericia...
4109,III.- ESTUDIOS SOLICITADOS\nA.- Fojas RMN homb...,estudios solicitados fojas hombros derecho cif...


In [16]:
print(dfnr.seccion_limpia[0])
print('Cantidad de palabras: ',(len(dfnr.seccion_limpia[0])))

proemio juez nacional hortas maria andrea perito medico oficio siguientes autos domicilio constituido calle austria piso dpto capital domicilio electronico mashortas cumplimiento dispuesto referente presentes autos examinado boiero franco ivan tomaron momento examen clinico actual consideraron capacidad generica particular puntos pericia solicitados partes
Cantidad de palabras:  358


In [17]:
text= 'Hola como estas che'

result = len(text.split())
result


4

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(longitudPalabras)
plt.title('Distribución de la longitud de palabras')

sns.despine();


In [ ]:
!pip install spacy==2.3

In [21]:
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups
def lematize(tok):
  lookups = Lookups()
  lookups.add_table("lemma_rules", {"noun": [["s", ""]]})
  lemmatizer = Lemmatizer(lookups)

  if tok.pos_== 'VERB':
    tok= tok.lemma_
  elif tok.pos_=='NOUN':
    tok = lemmatizer(tok.text, tok.pos_)[0]
  else:
    tok
  return tok

In [22]:
lemm = []
for sent in dfnr.seccion_limpia:
    tokens = nlp(sent)
    lem= [lematize(tok)for tok in tokens]
    lem = ' '.join(tok.text for tok in tokens)
    lemm.append(lem)

In [ ]:
#Lista de todas las secciones
lemm

In [46]:
# Cuenta la cantidad de palabras por sección y agrega esa columna "palabra" al dfnr

longitudPalabras = []
for palabra in lemm:
  longPalabras= len(palabra.split())

  longitudPalabras.append((longPalabras))

dfnr['palabras']= longitudPalabras
dfnr.head(30)


,parrafo,seccion_limpia,palabras
0,I-PROEMIO:\nSr. JUEZ NACIONAL:\nHORTAS MARIA A...,proemio juez nacional hortas maria andrea peri...,43
1,II-ANTECEDENTES DE INTERES MEDICO-LEGAL:\nlas ...,antecedentes interes medico legal encontraba c...,96
2,III-EXAMEN FISICO DEL ACTOR:\nIII-1 PROSOGRAFI...,examen fisico actor prosografia presenta exame...,539
3,IV-CONSIDERACIONES MEDICO-LEGALES:\nComenta el...,consideraciones medico legales comenta boiero ...,708
4,VII-CONCLUSIONES.\n1.\nLas secuelas en el tobi...,conclusiones secuelas tobillo izquierdo presen...,115
5,VII-INCAPACIDAD: el actor por las lesiones suf...,incapacidad actor lesiones sufridas presenta i...,23
6,VIII CONTESTACION DE LOS PUNTOS DE PERICIA:\nV...,contestacion puntos pericia actora contestado ...,9
7,IX-PETITORIO:\nHabiendo finalizado el trabajo ...,petitorio habiendo finalizado trabajo endado r...,45
8,II). PREAMBULO.\nSegun autos (texto de la dema...,preambulo autos texto demanda conforme manifes...,185
9,III). ANTECEDENTES DE INTERES MEDICO PRESENTE ...,antecedentes interes medico presente autos act...,129


In [45]:
dfnr.palabras.describe()

count    4111.000000
mean      246.966188
std       389.919628
min         1.000000
25%        42.000000
50%       103.000000
75%       304.500000
max      5970.000000
Name: palabras, dtype: float64

# Vectorización

In [ ]:
#Con min_df= 0.5 queda (4111,3)
from sklearn.feature_extraction.text import TfidfVectorizer
tfvectorizer = TfidfVectorizer( stop_words=stop_ws ,max_df=0.90, min_df=0.05, ngram_range=(1,1))
Xtf = tfvectorizer.fit_transform(lemm)

In [ ]:
Xtf.shape

In [ ]:
## reduccion de dimensionalidad : quitamos outliers? (valores atipicos)
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, algorithm='randomized', random_state=12)
normalizer = Normalizer(copy=False) # kmeans de sklearn no tiene dist coseno
lsa = make_pipeline(svd, normalizer)

X1 = lsa.fit_transform(Xtf)
X1.shape

In [ ]:
svd.explained_variance_ratio_.sum()

# Análisis de clusters: KMeans

In [ ]:
from sklearn.cluster import KMeans

Sum_of_squared_distances = []
K = range(1,20)
for k in K:
    km = KMeans(n_clusters=k, init='k-means++', n_init=10, verbose=False, random_state=12, algorithm = 'full')
    km = km.fit(X1)
    Sum_of_squared_distances.append(km.inertia_)

import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sumas de distancias al cuadrado')
plt.title('Método del codo para k óptimo')
plt.savefig('elbowmethod.jpg')
plt.show()

In [ ]:
km_clust = KMeans(n_clusters=10, init='k-means++', n_init=10, verbose=False, random_state=12)
km_clust.fit(X1)
dfnr['cluster']= km_clust.labels_
dfnr.head(15)

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x= 'cluster', data=dfnr)
plt.savefig('cluster_frec_kmeans.jpg')

In [ ]:
k=km_clust.n_clusters
BOLD = '\033[1m'
END = '\033[0m'
for i in range(k):

  print (BOLD+'\n Frases del Cluster {}\n'.format(i)+END,end='\n')
  print (dfnr[dfnr.cluster==i].seccion_limpia[0:10])

In [ ]:
def show_wordcloud(data, title = None):
    wordcloud = WordCloud().generate(str(data))
    fig = plt.figure(1, figsize=(6, 6))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)
    plt.imshow(wordcloud)
    plt.show()

#show_wordcloud(dfnr[dfnr.label==5].clean)

for i in range(km_clust.n_clusters):
  

  plt.title('Cluster {}'.format(i))

  show_wordcloud(dfnr[dfnr.cluster==i].seccion_limpia)

# Latent Dirichlet Allocation (LDA)

In [ ]:
#Con min_df=0.5 queda (4111,3)

from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, stop_words=stop_ws, ngram_range=(1,1), min_df=0.10)
tf = tf_vectorizer.fit_transform(lemm)
tf_feature_names = tf_vectorizer.get_feature_names()
tf.shape

In [ ]:
lda = LDA(n_components=10, max_iter=15, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda.components_.shape

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
# Se muestran las 5 palabras más frecuentes de cada tema que encontró LDA

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 5)

In [ ]:
doc_topic = lda.transform(tf)
lda_labels = [doc_topic[i].argmax() for i in range(doc_topic.shape[0])]
dfnr['lda'] = lda_labels
doc_topic.shape

In [ ]:
BOLD = '\033[1m'
END = '\033[0m'
for i in range(doc_topic.shape[1]):

  print (BOLD+'\n Frases del tema {}\n'.format(i)+END,end='\n')
  print (dfnr[dfnr.lda==i].seccion_limpia[0:10])

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x= 'lda', data=dfnr)
plt.savefig('frecs_lda.jpg')

# Reduccion de dimensionalidad: tSNE

In [ ]:
lda.components_.shape

In [ ]:
#  incrustación de vecinos estocásticos distribuidos en t

from sklearn.manifold import TSNE

def plot_tsne(model, label):
  X_tsne = TSNE(n_components=2).fit_transform(model)
  tsne_df = pd.DataFrame({'X':X_tsne[:,0],
                          'Y':X_tsne[:,1],
                          'Z':label})
  plt.figure(figsize=(20,10))
  sns.scatterplot(x="X", y="Y",
                hue='Z',
                palette= 'bright',
                legend='full',
                data=tsne_df);

In [ ]:
plot_tsne(X1, dfnr.cluster)# hacemos reduccion de dimensionalidad del modelo de k-means

In [ ]:
dfnr.lda

In [ ]:
plot_tsne(doc_topic, dfnr.lda)# hacemos reduccion de dimensionalidad de LDA

In [ ]:
X_tsne = TSNE(n_components=2, perplexity=200).fit_transform(doc_topic)
tsne_df = pd.DataFrame({'X':X_tsne[:,0],
                          'Y':X_tsne[:,1],
                          'Z':dfnr.lda})
plt.figure(figsize=(20,10))
sns.scatterplot(x="X", y="Y",
              hue='Z',
              palette= 'bright',
              legend='full',
              data=tsne_df);
plt.title('Reduccion de dimensionalidad del LDA con t-SNE')
plt.savefig('lda_tsne.jpg')

#Otra prueba de LDA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the vectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=0.10, max_df=0.95)
# fit_transform applies TF-IDF to clean texts - we save the array of vectors in X
X = vectorizer.fit_transform(lemm)
X.shape

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10)
lda.fit_transform(X)

In [ ]:
lda_dtf=lda.fit_transform(X)
lda_dtf.shape

In [ ]:
import numpy as np
sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(vectorizer.get_feature_names_out())

In [ ]:
!pip install mglearn

In [ ]:
import mglearn

mglearn.tools.print_topics(topics=range(8), feature_names=features, sorting=sorting, topics_per_chunk=5, n_words=8)

In [ ]:
!pip install umap-learn

#### Reducción con UMAP

In [ ]:
import umap

reducer = umap.UMAP(#random_state=42,
#                     init='random',
                    n_components=2,
                    n_neighbors=300,
                    min_dist=0.1,
#                     spread=2,
                    metric='euclidean',
                    verbose=True)

reducer.fit(X)

In [ ]:
reducer.transform(X)

In [ ]:
plt.figure(figsize=(40,40),facecolor='w')


# fig, ax = plt.subplots(facecolor='w')
plt.axis('off')
plt.scatter(reducer.embedding_[:, 0], reducer.embedding_[:, 1], s=2.5,c='red')


#     ax = plt.Axes(fig, [0., 0., 1., 1.])
#     ax.set_axis_off()

In [ ]:
import umap

reducer2 = umap.UMAP(#random_state=42,
#                     init='random',
                    n_components=2,
                    n_neighbors=300,
                    min_dist=0.1,
#                     spread=2,
                    metric='cosine',
                    verbose=True)

matrixT= reducer2.fit_transform(X)
matrixT

In [ ]:
plt.figure(figsize=(40,40),facecolor='w')


# fig, ax = plt.subplots(facecolor='w')
plt.axis('off')
plt.scatter(reducer2.embedding_[:, 0], reducer2.embedding_[:, 1], s=2.5,c='red')


#     ax = plt.Axes(fig, [0., 0., 1., 1.])
#     ax.set_axis_off()

In [ ]:
import umap

In [ ]:
mapper = umap.UMAP().fit(X)

In [ ]:
import umap.plot